In [39]:
import gzip
from rdflib.namespace import RDF, Namespace
from rdflib import URIRef
import pandas as pd

In [2]:
g = Dataset ()

file_path = "../data/graphs/https___druid.datalegend.net_RJM_Paramaribo-Ward-Registers-1828-1847_graphs_default.trig.gz"

with gzip.open(file_path, 'rt') as f:
    g.parse(f, format='trig')

print(f"Aantal triples: {len(g)}")

Aantal triples: 2078149


In [65]:
data = [] # Opslaan resultaten query

# Named Graph (een trig-bestand kan uit meerdere graven bestaan, wij hebben de volgende nodig:)
named_graph_uri = URIRef("https://druid.datalegend.net/RJM/Paramaribo-Ward-Registers-1828-1847/graphs/default")
named_graph = g.graph(named_graph_uri)

# Namespaces
SDO = Namespace("https://www.schema.org/")
PICO = Namespace("https://personsincontext.org/model#")
HDSC = Namespace("https://www.ru.nl/hdsc/example/")
PROV = Namespace("http://www.w3.org/ns/prov#")
ENSLAVED = Namespace("http://www.enslaved.org/1.0")
HSDC = Namespace("https://www.ru.nl/hdsc/example/")

# Op zoek naar Person Observations (in de named graph)
PERSON_OBSERVATION_URI = PICO.PersonObservation
for s, p, o in named_graph.triples((None, RDF.type, PERSON_OBSERVATION_URI)): # Itereer over alle observaties
    person_obs = {} # Dict om persoons observatie op teslaan (sla evt ID op voor later gebruik)

    for _, predicate, obj in named_graph.triples((s, None, None)):
        # Naam
        if predicate == SDO.name:
            person_obs['name'] = obj

        # Adres
        elif predicate == SDO.address:
            street_address = None
            location_name = None

            # Adreslabel is te verkrijgen via resulterende node
            for _, addr_predicate, addr_obj in named_graph.triples((obj, None, None)):
                if addr_predicate == SDO.streetAddress:
                    street_address = addr_obj
            person_obs['address_label'] = street_address

        # Lijst met enslaved (zou je eigenlijk meer informatie over willen, zie werkwijze adreslabel hierboven)
        elif predicate == HSDC.isEnslaverOf:
            person_obs['enslaver_of'] = obj

    data.append(person_obs) # Voeg dict voor deze persoonsobservatie toe aan resultatenlijst

df = pd.DataFrame(data) # Naar Pandas df voor verdere verwerking

In [67]:
df[['name', 'address_label', 'enslaver_of']]

,name,address_label,enslaver_of
0,"Esther D'Anavia, weduwe Monsantofils",Watermolenstraat 28,NaN
1,"Elisabeth Esther, weduwe Soesman",Steenbakkersgracht 96,NaN
2,Soesman,NaN,NaN
3,Adriana Esther Soesman,Steenbakkersgracht 96,https://www.ru.nl/hdsc/example/WR-Suriname/obs...
4,NaN,Steenbakkersgracht 96,NaN
...,...,...,...
144853,A.C.L. Smidt,Steenbakkersgracht 92b,NaN
144854,Johanna Wilhelmina van Fucht,Steenbakkersgracht 92b,NaN
144855,George Ferdinand Corbiere,Steenbakkersgracht 92b,NaN
144856,Carel Hendrik Corbiere,Steenbakkersgracht 92b,NaN
